In [2]:
import openai
import numpy as np
import pandas as pd

import transformers
import copy
import logging
from dataclasses import dataclass, field
from typing import Dict, Optional, Sequence

import utils
import re

In [6]:
heart_instructions = utils.heart2jinst(context, questions)
utils.jdump(heart_instructions, 'heart_data.json')


In [5]:
model_name_or_path = 'facebook/opt-125m'
cache_dir: str = None
optim: str = "adamw_torch"
model_max_length: int = 512
data_args = train.DataArguments(data_path='heart_data.json')

# make tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained(
        model_name_or_path,
        cache_dir=cache_dir,
        model_max_length=model_max_length,
        padding_side="right",
        use_fast=False,
    )

data_module = train.make_supervised_data_module(tokenizer=tokenizer, data_args=data_args)

In [7]:
data_module['train_dataset'][0]

{'input_ids': tensor([    2, 45943,    16,    41, 15741,    14,  7448,    10,  3685,     6,
         11153,    19,    41,  8135,    14,  1639,   617,  5377,     4, 21062,
            10,  1263,    14, 16574, 25830,     5,  2069,     4, 50118, 50118,
         48134, 41241,    35, 50118, 10836,    47,  6190,     5,  2511,     9,
             5, 12925, 11328, 10607,  1761,   116, 50118, 50118, 48134, 41327,
            35, 50118, 45152,   108, 14691, 13373,   128, 17779, 11328, 33425,
          3934,   128,  6423, 13373,   128, 22117, 18429,  5423,   108, 24303,
         50118, 50118, 48134, 19121,    35, 40022,  1253,     4,  1439,    16,
            10,   220,  2706, 10607,  1761,    14,  3291,   561,    41, 10764,
             9,  7193,   198,     5,   304,     9,  4687,     7,  1045,    23,
          3189,     4,     2]),
 'labels': tensor([ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -10

In [32]:
logging.warning("Formatting inputs...")

PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Response:"
    ),
}

prompt_input, prompt_no_input = PROMPT_DICT["prompt_input"], PROMPT_DICT["prompt_no_input"]

list_data_dict = utils.jload('alpaca_data_3000.json')

sources = [
    prompt_input.format_map(example) if example.get(
        "input", "") != "" else prompt_no_input.format_map(example)
    for example in list_data_dict
]
sources
targets = [
    f"{example['output']}{tokenizer.eos_token}" for example in list_data_dict]

# logging.warning("Tokenizing inputs... This may take some time...")
data_dict = train.preprocess(sources, targets, tokenizer)
sources[0].split(' ')
#targets[0]
len(data_dict['input_ids'][0])
input_ids = data_dict["input_ids"]
labels = data_dict["labels"]
input_ids[0]

tensor([    2, 45943,    16,    41, 15741,    14,  7448,    10,  3685,     4,
        21062,    10,  1263,    14, 16574, 25830,     5,  2069,     4, 50118,
        50118, 48134, 41241,    35, 50118, 31033,   130,  4965,    13,  4959,
         2245,     4, 50118, 50118, 48134, 19121,    35,   134,     4, 43800,
           10,  9320,  5626,     8,   146,   686,     7,   680,  2710,     9,
        12849,     8,  8942,     4,  1437, 50118,   176,     4, 30450,  4595,
            7,   489,   110,   809,  2171,     8,   670,     4,  1437, 50118,
          246,     4,  2315,   615,  3581,     8,  3014,    10,  4292,  3581,
         3078,     4,     2])